   ## WEEK3 HW 박상재

1번문제

In [ ]:
import torch
import torch.nn as nn

In [55]:
image=torch.Tensor([[[2,0,2,1,3,2],
                   [0,2,0,2,2,2],
                   [1,0,1,3,1,1],
                   [0,0,1,1,1,0],
                   [0,1,3,4,1,0],
                   [0,1,0,0,5,2]]])

In [56]:
image

tensor([[[2., 0., 2., 1., 3., 2.],
         [0., 2., 0., 2., 2., 2.],
         [1., 0., 1., 3., 1., 1.],
         [0., 0., 1., 1., 1., 0.],
         [0., 1., 3., 4., 1., 0.],
         [0., 1., 0., 0., 5., 2.]]])

In [57]:
image_conv=torch.Tensor([[[4,0,4,3,5,2],
                        [0,5,3,5,4,5],
                        [1,1,4,4,3,3],
                        [1,4,5,3,4,1],
                        [1,1,3,10,4,1],
                        [0,1,1,3,9,3]]])

In [58]:
image_conv

tensor([[[ 4.,  0.,  4.,  3.,  5.,  2.],
         [ 0.,  5.,  3.,  5.,  4.,  5.],
         [ 1.,  1.,  4.,  4.,  3.,  3.],
         [ 1.,  4.,  5.,  3.,  4.,  1.],
         [ 1.,  1.,  3., 10.,  4.,  1.],
         [ 0.,  1.,  1.,  3.,  9.,  3.]]])

In [59]:
pool=torch.nn.MaxPool2d(2)

In [60]:
out=pool(image_conv)

In [54]:
out

tensor([[[ 5.,  5.,  5.],
         [ 4.,  5.,  4.],
         [ 1., 10.,  9.]]])

2번문제

In [93]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device ='cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [94]:
#학습에 사용되는 parameters 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [95]:
#MNIST 데이터셋 가져오기
mnist_train = dsets.MNIST(root='MNIST_data/',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [96]:
#dataset loader 만들기
data_loader=torch.utils.data.DataLoader(dataset=mnist_train,
                                       batch_size=batch_size,
                                       shuffle=True,
                                       drop_last=True)

2-1 Layer1의 Kernel size를 4로 늘리고, Layer3의 stride를 2로 늘려라

In [117]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        
        self.keep_prob = 0.5
        
        self.layer1=torch.nn.Sequential(
            torch.nn.Conv2d(1,32,kernel_size=4,stride=1,padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2))
        
        self.layer2=torch.nn.Sequential(
            torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2))
        
        self.layer3=torch.nn.Sequential(
            torch.nn.Conv2d(64,128,kernel_size=3,stride=2,padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2,padding=1))
        

        self.fc1=torch.nn.Linear(2*2*128,625,bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        
        self.layer4=torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1-self.keep_prob))
        
        self.fc2= torch.nn.Linear(625,10,bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self,x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=self.layer3(out)
        out=out.view(out.size(0),-1) #fc를 위해 faltten
        out=self.layer4(out)
        out=self.fc2(out)
        return out

2-2 Layer3의 Output size를 구하여라

In [118]:
input=torch.FloatTensor(1,1,28,28)
conv1=torch.nn.Conv2d(1,32,kernel_size=4,stride=1,padding=1)
conv2=torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1)
conv3=torch.nn.Conv2d(64,128,kernel_size=3,stride=2,padding=1)
pool=torch.nn.MaxPool2d(kernel_size=2,stride=2)
pool_pad=torch.nn.MaxPool2d(kernel_size=2,stride=2,padding=1)

out1=conv1(input)
out2=pool(out1)
out2.shape
out3=conv2(out2)
out4=pool(out3)
out5=conv3(out4)
out6=pool_pad(out5)
print(out1.shape,'\n',
      out2.shape,'\n',
      out3.shape,'\n',
      out4.shape,'\n',
      out5.shape,'\n',
      out6.shape,'\n',)

torch.Size([1, 32, 27, 27]) 
 torch.Size([1, 32, 13, 13]) 
 torch.Size([1, 64, 13, 13]) 
 torch.Size([1, 64, 6, 6]) 
 torch.Size([1, 128, 3, 3]) 
 torch.Size([1, 128, 2, 2]) 



In [119]:
model=CNN()
#loss function, optimizer 선택
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [120]:
total_batch=len(data_loader)
print('Learning started')
for epoch in range(training_epochs):
    avg_cost=0
    
    for X,Y in data_loader:
        #이미지는 28x28이므로 reshape 해줄 필요가 없다. 크기 다르면 X.view(-1,28*28)
        #label is not one-hot enconded
        X=X
        Y=Y
        
        optimizer.zero_grad()
        hypothesis=model(X)
        cost=criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
        
        avg_cost+=cost/total_batch
        
    print('[Epoch: {:>4}] cost={:>.9}'.format(epoch+1,avg_cost))
print('Learning Finished')

Learning started
[Epoch:    1] cost=0.251809567
[Epoch:    2] cost=0.0678042546
[Epoch:    3] cost=0.047202073
[Epoch:    4] cost=0.0365661718
[Epoch:    5] cost=0.0296426006
[Epoch:    6] cost=0.0242779981
[Epoch:    7] cost=0.0212091766
[Epoch:    8] cost=0.0195036866
[Epoch:    9] cost=0.0163822584
[Epoch:   10] cost=0.0158765558
[Epoch:   11] cost=0.0139919017
[Epoch:   12] cost=0.0114730056
[Epoch:   13] cost=0.0116501097
[Epoch:   14] cost=0.0105896313
[Epoch:   15] cost=0.0110794595
Learning Finished


#기존 모델의 cost
 Learning started
 
 [Epoch:    1] cost=0.316445887
 
 [Epoch:    2] cost=0.0742798969
 
 [Epoch:    3] cost=0.0542666502
 
 [Epoch:    4] cost=0.0439512655
 
 [Epoch:    5] cost=0.0363897085
 
 [Epoch:    6] cost=0.0307028312
 
 [Epoch:    7] cost=0.0256947763
 
 [Epoch:    8] cost=0.0223205276
 
 [Epoch:    9] cost=0.0186817795
 
 [Epoch:   10] cost=0.0156495944
 
 [Epoch:   11] cost=0.0137053365
 
 [Epoch:   12] cost=0.0115955742
 
 [Epoch:   13] cost=0.00855956785
 
 [Epoch:   14] cost=0.00930883642
 
 [Epoch:   15] cost=0.00790862832
 Learning Finished

In [122]:
with torch.no_grad():
    X_test=mnist_test.test_data.view(len(mnist_test),1,28,28).float()
    Y_test=mnist_test.test_labels
    
    prediction=model(X_test)
    correct_prediction=torch.argmax(prediction,1)== Y_test
    accuracy=correct_prediction.float().mean()
    print('Accuracy:',accuracy.item())

C:\Users\psj\anaconda3\envs\esc\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\psj\anaconda3\envs\esc\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9887999892234802


기존 모델의 accuracy

Accuracy: 0.9848999977111816

2-3 Cost와 accuracy가 기존 CNN모델에 비해 어떻게 변할 것인가?

더 Deep 하게 쌓았기 때문에 accuracy는 높아질 것이라고 생각하고
cost는 기존 모델에 의해 underfitting되었는지 overfitting 되었는지 모르기 때문에 정확히 예측 할 수 없었다.

2-4 코딩하여 cost와 accuracy를 확인해보자!

cost는 증가 했지만 accuracy도 증가했다.